In [1]:
#!/bin/bash

# Download original ShapeNetPart dataset (around 1GB)
!wget --no-check-certificate https://shapenet.cs.stanford.edu/ericyi/shapenetcore_partanno_v0.zip
!unzip shapenetcore_partanno_v0.zip
!rm shapenetcore_partanno_v0.zip

# Download HDF5 for ShapeNet Part segmentation (around 346MB)
!wget --no-check-certificate https://shapenet.cs.stanford.edu/media/shapenet_part_seg_hdf5_data.zip
!unzip shapenet_part_seg_hdf5_data.zip
!rm shapenet_part_seg_hdf5_data.zip



Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  inflating: PartAnnotation/02691156/points/a19a5a459d234919c1ccec171a275967.pts  
  inflating: PartAnnotation/02691156/points/7dc614fd957186cf216b327569dd806e.pts  
  inflating: PartAnnotation/02691156/points/e9bae38bc2083d0bb4d73e4449062b04.pts  
  inflating: PartAnnotation/02691156/points/444a58c4c6bfcd6c7a571401e6605fc2.pts  
  inflating: PartAnnotation/02691156/points/7662fbb435dc1d0760bb9786c97a649e.pts  
  inflating: PartAnnotation/02691156/points/a85dbb5bb889881419bf80b167cee3a3.pts  
  inflating: PartAnnotation/02691156/points/7fce28d597b3137cf24f8f5f80968450.pts  
  inflating: PartAnnotation/02691156/points/f2171bb2d715140c8b96ae1a0a8b84ec.pts  
  inflating: PartAnnotation/02691156/points/7a794db8180858fe90916c8815b5c43.pts  
  inflating: PartAnnotation/02691156/points/b509fbbe99df854f0478431b5ad57db.pts  
  inflating: PartAnnotation/02691156/points/3109b3740ac1a056df55310efbf4c0e4.pts  
  inflating

In [3]:
!pip install plyfile

In [5]:
import argparse 
import math
import h5py
import numpy as np
import socket
import importlib
import matplotlib.pyplot as plt
import os
import sys


import provider
import math
import random
import data_utils
import time

import torch
from torch import nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader


from model import RandPointCNN
from util_funcs import knn_indices_func_gpu, knn_indices_func_cpu
from util_layers import Dense

In [8]:



random.seed(0)
dtype = torch.cuda.FloatTensor

# Load Hyperparameters
gpu=0
model='pointnet_cls'             
log_dir='log'
num_point=1024
max_epoch=2
batch_size=32
learning_rate=0.001
momentum=0.9
optimizer='adam'
decay_step=200000
decay_rate=0.7


NUM_POINT = num_point
LEARNING_RATE = learning_rate
GPU_INDEX = gpu
MOMENTUM = momentum
       
MAX_NUM_POINT = 2048

DECAY_STEP = decay_step
DECAY_RATE = decay_rate
BN_INIT_DECAY = 0.5
BN_DECAY_DECAY_RATE = 0.5
BN_DECAY_DECAY_STEP = float(DECAY_STEP)
BN_DECAY_CLIP = 0.99


LEARNING_RATE_MIN = 0.00001
        
NUM_CLASS = 40
BATCH_SIZE = batch_size #32
NUM_EPOCHS = max_epoch
jitter = 0.01
jitter_val = 0.01

rotation_range = [0, math.pi / 18, 0, 'g']
rotation_rage_val = [0, 0, 0, 'u']
order = 'rxyz'

scaling_range = [0.05, 0.05, 0.05, 'g']
scaling_range_val = [0, 0, 0, 'u']

In [9]:
class modelnet40_dataset(Dataset):

    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, i):
        return self.data[i], self.labels[i]

In [10]:
# C_in, C_out, D, N_neighbors, dilution, N_rep, r_indices_func, C_lifted = None, mlp_width = 2
# (a, b, c, d, e) == (C_in, C_out, N_neighbors, dilution, N_rep)
# Abbreviated PointCNN constructor.
AbbPointCNN = lambda a, b, c, d, e: RandPointCNN(a, b, 3, c, d, e, knn_indices_func_gpu)

In [11]:
class Classifier(nn.Module):

    def __init__(self):
        super(Classifier, self).__init__()

        self.pcnn1 = AbbPointCNN(3, 32, 8, 1, -1)
        self.pcnn2 = nn.Sequential(
            AbbPointCNN(32, 64, 8, 2, -1),
            AbbPointCNN(64, 96, 8, 4, -1),
            AbbPointCNN(96, 128, 12, 4, 120),
            AbbPointCNN(128, 160, 12, 6, 120)
        )

        self.fcn = nn.Sequential(
            Dense(160, 128),
            Dense(128, 64, drop_rate=0.5),
            Dense(64, NUM_CLASS, with_bn=False, activation=None)
        )

    def forward(self, x):
        x = self.pcnn1(x)
        if False:
            print("Making graph...")
            k = make_dot(x[1])

            print("Viewing...")
            k.view()
            print("DONE")

            assert False
        x = self.pcnn2(x)[1]  # grab features

        logits = self.fcn(x)
        logits_mean = torch.mean(logits, dim=1)
        return logits_mean


print("------Building model-------")
model = Classifier().cuda()

------Building model-------


In [13]:

print("------Successfully Built model-------")


optimizer = torch.optim.SGD(model.parameters(), lr = 0.01, momentum = 0.9)
loss_fn = nn.CrossEntropyLoss()

global_step = 1



TRAIN_FILES = provider.getDataFiles("hdf5_data/train_hdf5_file_list.txt")
TEST_FILES = provider.getDataFiles('hdf5_data/val_hdf5_file_list.txt')

losses = []
accuracies = []

------Successfully Built model-------


In [ ]:
for epoch in range(1, NUM_EPOCHS+1):
    train_file_idxs = np.arange(0, len(TRAIN_FILES))
    np.random.shuffle(train_file_idxs)

    for fn in range(len(TRAIN_FILES)):
        #log_string('----' + str(fn) + '-----')
        current_data, current_label = provider.loadDataFile("hdf5_data/"+TRAIN_FILES[train_file_idxs[fn]])
        current_data = current_data[:, 0:NUM_POINT, :]

        current_data, current_label, _ = provider.shuffle_data(current_data, np.squeeze(current_label))
        current_label = np.squeeze(current_label)

        file_size = current_data.shape[0]
        num_batches = file_size // BATCH_SIZE

        total_correct = 0
        total_seen = 0
        loss_sum = 0

        if epoch > 1:
            LEARNING_RATE *= decay_rate ** (global_step // decay_steps)
            if LEARNING_RATE > LEARNING_RATE_MIN:
                print("NEW LEARNING RATE:", LEARNING_RATE)
                optimizer = torch.optim.SGD(model.parameters(), lr = LEARNING_RATE, momentum = 0.9)

        for batch_idx in range(num_batches):
            start_idx = batch_idx * BATCH_SIZE
            end_idx = (batch_idx + 1) * BATCH_SIZE

            # Lable
            label = current_label[start_idx:end_idx]
            label = torch.from_numpy(label).long()
            label = Variable(label, requires_grad=False).cuda()
            # Augment batched point clouds by rotation and jittering
            rotated_data = provider.rotate_point_cloud(current_data[start_idx:end_idx, :, :])
            jittered_data = provider.jitter_point_cloud(rotated_data) # P_Sampled
            P_sampled = jittered_data
            F_sampled = np.zeros((BATCH_SIZE, NUM_POINT, 0))
            optimizer.zero_grad()

            t0 = time.time()
            P_sampled = torch.from_numpy(P_sampled).float()
            P_sampled = Variable(P_sampled, requires_grad=False).cuda()

            #F_sampled = torch.from_numpy(F_sampled)

            out = model((P_sampled, P_sampled))
            loss = loss_fn(out, label)
            loss.backward()
            optimizer.step()
            print("epoch: "+str(epoch) + "   loss: "+str(loss.item()))
            if global_step % 25 == 0:
                loss_v = loss.data.item()
                print("Loss:", loss_v)
            else:
                loss_v = 0
            global_step += 1

/content/provider.py:91: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  f = h5py.File(h5_filename)


epoch: 1   loss: 3.0246691703796387
Loss: 3.0246691703796387
epoch: 1   loss: 2.8239316940307617
